In [6]:
# -*- coding: utf-8 -*-
from __future__ import annotations

import os
import json
from typing import Any, Dict, Iterable, Optional

import requests
from dotenv import load_dotenv

load_dotenv()

LAW_SERVICE_URL = "http://www.law.go.kr/DRF/lawService.do"

MST = "276291"
EFYD = "20260102"
CHR_CLS_CD = "010202"  # 한글 고정


def require_oc() -> str:
    oc = os.getenv("LAW_OC", "").strip()
    if not oc:
        raise RuntimeError("LAW_OC가 비어있음 (.env 또는 환경변수 확인)")
    return oc


def iter_dicts(obj: Any) -> Iterable[Dict[str, Any]]:
    """JSON 전체를 재귀로 돌며 dict를 모두 yield"""
    if isinstance(obj, dict):
        yield obj
        for v in obj.values():
            yield from iter_dicts(v)
    elif isinstance(obj, list):
        for it in obj:
            yield from iter_dicts(it)


def first_n_article_dicts(body: Any, n: int = 2) -> list[Dict[str, Any]]:
    arts = []
    for d in iter_dicts(body):
        if isinstance(d, dict) and "조문번호" in d:
            arts.append(d)
            if len(arts) >= n:
                break
    return arts


def main():
    oc = require_oc()

    params = {
        "OC": oc,
        "target": "eflaw",
        "type": "JSON",
        "MST": MST,
        "efYd": EFYD,
        "chrClsCd": CHR_CLS_CD,
    }

    r = requests.get(LAW_SERVICE_URL, params=params, timeout=30)

    print("[HTTP] status:", r.status_code)
    print("[HTTP] content-type:", r.headers.get("content-type", ""))
    print("[HTTP] url:", r.url)

    r.raise_for_status()

    # JSON 파싱
    try:
        data = r.json()
    except Exception:
        print("\n[ERROR] JSON 파싱 실패. 응답 앞부분(1000자):")
        print((r.text or "")[:1000])
        return

    # 구조 확인용: 최상위 키만 출력
    if isinstance(data, dict):
        print("\n[DEBUG] top-level keys:", list(data.keys())[:30])

    # 첫 조문 찾기
    arts = first_n_article_dicts(data, n=2)
    if not arts:
        print("조문 못 찾음 ...")
        return
    for art in arts:
        jo_no = art.get("조문번호")
        jo_title = art.get("조문제목")
        jo_text = art.get("조문내용")

        print("\n========== ARTICLE ==========")
        print("조문번호:", jo_no)
        print("조문제목:", jo_title)
        print("조문내용:", (jo_text or "")[:500])

    # 조문내용이 비면 항에서 찾기
    if not jo_text:
        hang_list = art.get("항")
        if isinstance(hang_list, list) and hang_list:
            first_hang = hang_list[0]
            print("\n------ FIRST PARAGRAPH ------")
            print("항번호:", first_hang.get("항번호"))
            print("항내용:", (first_hang.get("항내용") or "")[:500])
        else:
            print("\n[INFO] 이 조문 dict 안에 '항' 리스트도 없음. (응답 구조가 다른 위치에 있을 수 있음)")


if __name__ == "__main__":
    main()


[HTTP] status: 200
[HTTP] content-type: application/json;charset=UTF-8
[HTTP] url: http://www.law.go.kr/DRF/lawService.do?OC=dgjjanggu&target=eflaw&type=JSON&MST=276291&efYd=20260102&chrClsCd=010202

[DEBUG] top-level keys: ['법령']

========== ARTICLE ==========
조문번호: 1
조문제목: 목적
조문내용: 제1조(목적) 이 법은 주거용 건물의 임대차(賃貸借)에 관하여 「민법」에 대한 특례를 규정함으로써 국민 주거생활의 안정을 보장함을 목적으로 한다.

========== ARTICLE ==========
조문번호: 2
조문제목: 적용 범위
조문내용: 제2조(적용 범위) 이 법은 주거용 건물(이하 "주택"이라 한다)의 전부 또는 일부의 임대차에 관하여 적용한다. 그 임차주택(賃借住宅)의 일부가 주거 외의 목적으로 사용되는 경우에도 또한 같다.
